# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,30.45,49,0,4.12,US,1721590067
1,1,polyarnyy,69.1989,33.4478,18.96,66,92,1.70,RU,1721590068
2,2,adamstown,-25.0660,-130.1015,18.80,66,95,8.88,PN,1721590069
3,3,yellowknife,62.4560,-114.3525,26.34,42,75,3.09,CA,1721590070
4,4,port alfred,-33.5906,26.8910,13.04,68,8,2.54,ZA,1721590071


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[
    (city_data_df["Max Temp"] < 30) \
    & (city_data_df["Max Temp"] > 20) \
    & (city_data_df["Wind Speed"] < 8) \
    & (city_data_df["Humidity"] < 50) \
    & (city_data_df["Cloudiness"] <= 10)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,qarazhal,48.0078,70.7903,23.24,46,0,2.86,KZ,1721590116
122,122,parry sound,45.3501,-80.0330,26.03,34,0,3.05,CA,1721590210
123,123,aspen,39.1911,-106.8175,25.47,25,0,5.66,US,1721589962
154,154,buka,40.8108,69.1986,25.49,27,0,3.12,UZ,1721590247
174,174,lugovoye,42.9094,72.7178,22.28,44,0,1.64,KZ,1721590273
177,177,qorveh,35.1664,47.8045,22.58,26,0,2.98,IR,1721590277
189,189,fasa,28.9383,53.6482,29.58,17,0,1.03,IR,1721590290
208,208,posse,-14.0931,-46.3694,27.67,28,5,3.72,BR,1721590312
232,232,laojunmiao,39.8333,97.7333,21.78,28,0,4.23,CN,1721590339
245,245,mary,37.0000,62.5000,29.99,18,0,3.63,TM,1721590354


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
43,qarazhal,KZ,48.0078,70.7903,46,
122,parry sound,CA,45.3501,-80.0330,34,
123,aspen,US,39.1911,-106.8175,25,
154,buka,UZ,40.8108,69.1986,27,
174,lugovoye,KZ,42.9094,72.7178,44,
177,qorveh,IR,35.1664,47.8045,26,
189,fasa,IR,28.9383,53.6482,17,
208,posse,BR,-14.0931,-46.3694,28,
232,laojunmiao,CN,39.8333,97.7333,28,
245,mary,TM,37.0000,62.5000,18,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
qarazhal - nearest hotel: No hotel found
parry sound - nearest hotel: Mid Town Hotel
aspen - nearest hotel: Hotel Jerome, An Auberge Resort Collection
buka - nearest hotel: Brez
lugovoye - nearest hotel: No hotel found
qorveh - nearest hotel: تالار شادی
fasa - nearest hotel: هتل جهانگردی فسا
posse - nearest hotel: Palmeiras Hotel
laojunmiao - nearest hotel: No hotel found
mary - nearest hotel: No hotel found
elazig - nearest hotel: DoubleTree by Hilton Hotel Elazig
kyzylorda - nearest hotel: Нур
los alamos - nearest hotel: Comfort Inn & Suites Los Alamos
aricak - nearest hotel: No hotel found
tres isletas - nearest hotel: Hotel España
sandnessjoen - nearest hotel: Scandic Syv Søstre
nahrin - nearest hotel: قصر عروسی صدف
dzhalagash - nearest hotel: No hotel found
al qaryatayn - nearest hotel: No hotel found
chardara - nearest hotel: Бақдаулет
north platte - nearest hotel: Howard Johnson Inn - North Platte
yulinshi - nearest hotel: 榆溪大酒店
cruzeiro - nearest hotel: Ch

,City,Country,Lat,Lng,Humidity,Hotel Name
43,qarazhal,KZ,48.0078,70.7903,46,No hotel found
122,parry sound,CA,45.3501,-80.0330,34,Mid Town Hotel
123,aspen,US,39.1911,-106.8175,25,"Hotel Jerome, An Auberge Resort Collection"
154,buka,UZ,40.8108,69.1986,27,Brez
174,lugovoye,KZ,42.9094,72.7178,44,No hotel found
177,qorveh,IR,35.1664,47.8045,26,تالار شادی
189,fasa,IR,28.9383,53.6482,17,هتل جهانگردی فسا
208,posse,BR,-14.0931,-46.3694,28,Palmeiras Hotel
232,laojunmiao,CN,39.8333,97.7333,28,No hotel found
245,mary,TM,37.0000,62.5000,18,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=1,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)